# Milestone 2

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import os, zipfile , json , random, requests
import re
from pathlib import Path
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import  Sequential

2025-04-21 21:51:10.344529: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745272270.601879      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745272270.674422      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Explorting dataset:

In [35]:

def is_kaggle():
    # Kaggle kernels always set this env var
    return 'KAGGLE_URL_BASE' in os.environ

def is_colab():
    return (not is_kaggle()) and os.path.exists('/content')

def maybe_mount_drive():
    if is_colab():
        from google.colab import drive
        if not os.path.isdir('/content/drive'):
            drive.mount('/content/drive')

def get_data_path():
    if is_kaggle():
        return '/kaggle/input/squad-2-0/'
    elif is_colab():
        return '/content/drive/MyDrive/SQuAD'
    else:
        return './data/'
def get_model_dir():
    if is_colab():
        model_dir = '/content/drive/MyDrive/models'
    elif is_kaggle():
        model_dir = '/kaggle/working/models'
    else:
        model_dir = './models'
    os.makedirs(model_dir, exist_ok=True)
    return model_dir

In [3]:
dataset_dir = get_data_path()
tokenizer_phase_2()
os.makedirs(dataset_dir, exist_ok=True)

In [4]:
file_path = os.path.join(dataset_dir, 'train-v2.0.json')

In [5]:
with open(file_path, 'r', encoding='utf-8') as f:
    squad = json.load(f)

In [6]:
records = []
for article in squad['data']:
    for para in article['paragraphs']:
        ctx = para['context']
        for qa in para['qas']:
            answers = [a['text'] for a in qa.get('answers', [])]
            starts  = [a['answer_start'] for a in qa.get('answers', [])]
            ends    = [s + len(t) for s,t in zip(starts, answers)]
            records.append({
                'question': qa['question'],
                'answers': answers,
                'context': ctx,
                'answer_start': starts,
                'answer_end': ends
            })



In [7]:
df = pd.DataFrame(records)
df.head()

,question,answers,context,answer_start,answer_end
0,When did Beyonce start becoming popular?,[in the late 1990s],Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,[269],[286]
1,What areas did Beyonce compete in when she was...,[singing and dancing],Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,[207],[226]
2,When did Beyonce leave Destiny's Child and bec...,[2003],Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,[526],[530]
3,In what city and state did Beyonce grow up?,"[Houston, Texas]",Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,[166],[180]
4,In which decade did Beyonce become famous?,[late 1990s],Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,[276],[286]


In [8]:
print("Total QA pairs:", len(df))

Total QA pairs: 130319


In [42]:
#shuffling
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# only working on subset of 15k row
df_subset = df.head(15000).copy().reset_index(drop=True)

print("Subset size:", df_subset.shape)
df_subset.head()

Subset size: (15000, 5)


,question,answers,context,answer_start,answer_end
0,What are built in many places through the cities?,[tablados],"During the celebration, theaters called tablad...",[40],[48]
1,What was Tamuramaro's military title?,[Sei-i Taishōgun],Although Kammu had abandoned universal conscri...,[326],[341]
2,What lies to the west of Santa Monica?,[],Santa Monica is a beachfront city in western L...,[],[]
3,How many seasons was Jennifer Lopez a judge on...,[5],"Fox announced on May 11, 2015 that the fifteen...",[28],[29]
4,Which two-time Super Bowl winner graduated fro...,[Jim McMahon],A number of BYU alumni have found success in p...,[852],[863]


## Data Cleaning

Dropping rows where answers are empty

In [43]:
df_subset = df_subset[df_subset['answers'].map(len) > 0].reset_index(drop=True)
print("Rows remaining after drop:", len(df_subset))

Rows remaining after drop: 10018


Removing Extra Whitespaces

In [44]:
def collapse_whitespace(s):
    if isinstance(s, str):
        return re.sub(r'\s+', ' ', s.strip())
    return s

In [45]:
for col in ['question', 'context', 'answers']:
    if col in df_subset.columns:
        df_subset[col] = df_subset[col].apply(collapse_whitespace)

**Lets explore the length of the sequences which will determine some hyperparameters in training the models**

In [46]:
df_subset['question'].str.len().max()

270

In [47]:
df_subset['context'].str.len().max()

3385

We just turn the array of the answers to a string since none have multiple answers

In [48]:
df_subset['answers']= df_subset['answers'].apply(lambda x: x[0])

In [49]:
df_subset['answers'].str.len().max()

161

## Embeddings

In [50]:
!pip install --quiet gensim

**Tokenizer for phase 1 only**

In [51]:
tokenizer_phase_1 = Tokenizer(
    num_words=20000,
    oov_token='[UNK]',
    filters='''!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'''
)
tokenizer_phase_1.fit_on_texts(df_subset["question"].tolist()+df_subset["answers"].tolist())

q_seqs = tokenizer_phase_1.texts_to_sequences(df_subset['question'])
a_seqs = tokenizer_phase_1.texts_to_sequences(df_subset['answers'])



In [52]:
vocab_size = len(tokenizer_phase_1.word_index)
print("Total unique tokens:", vocab_size)

Total unique tokens: 18773


**Tokenizer for phase 2**

In [97]:
def truncate_context(context: str, ans_start: int, ans_end: int, max_len: int) -> str:
    """
    Return a substring of `context` of length up to max_len characters,
    centered on the character span [ans_start:ans_end], adjusted to word boundaries.
    """
    ans_len = ans_end - ans_start
    extra   = max_len - ans_len
    pre     = extra // 2
    post    = extra - pre

    # ideal window
    start = ans_start - pre
    end   = ans_end   + post

    # shift if off left edge
    if start < 0:
        start = 0
        end   = min(max_len, len(context))

    # shift if off right edge
    if end > len(context):
        end   = len(context)
        start = max(0, len(context) - max_len)

    # adjust start backward to nearest whitespace to avoid cutting a word
    if start > 0 and not context[start].isspace():
        m = re.search(r'\s', context[:start][::-1])
        if m:
            # position of last whitespace before start
            start = start - m.start()

    # adjust end forward to nearest whitespace to avoid cutting a word
    if end < len(context) and not context[end].isspace():
        m = re.search(r'\s', context[end:])
        if m:
            end = end + m.start()

    # final slice
    return context[start:end]

def build_truncated_context(df, max_len: int):
    """
    Returns a list of truncated context strings for each row in df,
    preserving at least the answer span and cutting only at word boundaries.
    """
    contexts = []
    for ctx, starts, ends in zip(df['context'], df['answer_start'], df['answer_end']):
        # pick the first span
        s = starts[0]
        e = ends[0]

        window = truncate_context(ctx, s, e, max_len)
        contexts.append(window)

    return contexts

In [99]:

questions = df_subset['question'].astype(str).tolist()
answers   = df_subset['answers'].astype(str).tolist()
contexts  = build_truncated_context(df_subset, 2000)

tokenizer_phase_2 = Tokenizer(
    num_words=50000,
    oov_token='[UNK]',
    filters='''!"#$%&()*+,-./:;<=>?@\\^_`{|}~\t\n'''
)
tokenizer_phase_2.fit_on_texts(questions + answers + contexts + ['[SEP]'])


**Adding sep sos and eos tag**

In [100]:
sep_token = "[SEP]"
sep_id = tokenizer_phase_2.word_index.get(sep_token)
if sep_id is None:
    sep_id = tokenizer_phase_2.num_words - 1
    occupant = tokenizer_phase_2.index_word.get(sep_id)
    if occupant:
        del tokenizer_phase_2.word_index[occupant] 
    tokenizer_phase_2.word_index[sep_token] = sep_id
    tokenizer_phase_2.index_word[sep_id]  = sep_token
print("✅ SEP_TOKEN id =", sep_id)


for token, offset in {"[SOS]":1, "[EOS]":2}.items():
    tok_id = tokenizer_phase_2.word_index.get(token)
    if tok_id is None:
        new_id = tokenizer_phase_2.num_words - offset - 1
        occupant = tokenizer_phase_2.index_word.get(new_id)
        if occupant:
            del tokenizer_phase_2.word_index[occupant]
        tokenizer_phase_2.word_index[token]      = new_id
        tokenizer_phase_2.index_word[new_id]     = token
        print(f"✅ {token} injected at id {new_id}")
    else:
        print(f"✅ {token} already at id {tok_id}")

sep_id = tokenizer_phase_2.word_index['[SEP]']
sos_id = tokenizer_phase_2.word_index['[SOS]']
eos_id = tokenizer_phase_2.word_index['[EOS]']
pad_id = 0

print("Special IDs:", {'SEP':sep_id, 'SOS':sos_id, 'EOS':eos_id, 'PAD':pad_id})






✅ SEP_TOKEN id = 49999
✅ [SOS] injected at id 49998
✅ [EOS] injected at id 49997
Special IDs: {'SEP': 49999, 'SOS': 49998, 'EOS': 49997, 'PAD': 0}


In [101]:
q_seqs = tokenizer_phase_2.texts_to_sequences(questions)
c_seqs = tokenizer_phase_2.texts_to_sequences(contexts)
a_raw  = tokenizer_phase_2.texts_to_sequences(answers)
a_seqs = [[sos_id] + seq + [eos_id] for seq in a_raw]

MAX_Q_LEN  = max(len(s) for s in q_seqs)
MAX_C_LEN  = max(len(s) for s in c_seqs)
MAX_A_LEN  = max(len(s) for s in a_seqs)
MAX_ENCODER_LEN = MAX_Q_LEN + 1 + MAX_C_LEN

print("Lengths:", {'Q':MAX_Q_LEN, 'C':MAX_C_LEN, 'A':MAX_A_LEN, 'Enc':MAX_ENCODER_LEN})

Lengths: {'Q': 40, 'C': 383, 'A': 32, 'Enc': 424}


**Add padding to the encoder and decoder inputs**

In [102]:
enc_seqs = [q + [sep_id] + c for q, c in zip(q_seqs, c_seqs)]
encoder_inputs = pad_sequences(
    enc_seqs,
    maxlen=MAX_ENCODER_LEN,
    padding='post',
    truncating='post',
    value=pad_id
)

a_padded = pad_sequences(
    a_seqs,
    maxlen=MAX_A_LEN,
    padding='post',
    truncating='post',
    value=pad_id
)

**Split the data**

In [103]:
decoder_inputs  = a_padded[:, :-1]  # begins with SOS
decoder_targets = a_padded[:,  1:]  # ends with EOS

print("encoder_inputs:", encoder_inputs.shape)
print("decoder_inputs:", decoder_inputs.shape)
print("decoder_targets:", decoder_targets.shape)
assert decoder_inputs.shape == decoder_targets.shape

(enc_tr, enc_val,
 decin_tr, decin_val,
 dectar_tr, dectar_val) = train_test_split(
    encoder_inputs,
    decoder_inputs,
    decoder_targets,
    test_size=0.1,
    random_state=42
)

encoder_inputs: (10018, 424)
decoder_inputs: (10018, 31)
decoder_targets: (10018, 31)


**Load gloVe dictionary**

In [104]:
def prepare_glove(target_dim=100, work_subdir='glove',
                  input_dataset_slug='glove6b',
                  download_url='http://nlp.stanford.edu/data/glove.6B.zip'):
    maybe_mount_drive()

    if is_kaggle():
        work_dir = f'/kaggle/working/{work_subdir}'
        uploaded_zip = f'/kaggle/input/{input_dataset_slug}/glove.6B.zip'
    elif is_colab():
        work_dir = f'/content/drive/MyDrive/{work_subdir}'
        uploaded_zip = None
    else:
        work_dir = f'./data/{work_subdir}'
        uploaded_zip = None

    os.makedirs(work_dir, exist_ok=True)

    target_file = f'glove.6B.{target_dim}d.txt'
    txt_path = os.path.join(work_dir, target_file)
    zip_path = os.path.join(work_dir, os.path.basename(download_url))

    if os.path.exists(txt_path):
        return txt_path

    if is_kaggle() and uploaded_zip and os.path.exists(uploaded_zip):
        zip_path = uploaded_zip
    else:
        if requests is None:
            raise RuntimeError("`requests` not available; offline mode")
        with requests.get(download_url, stream=True) as r, open(zip_path, 'wb') as f:
            r.raise_for_status()
            for chunk in r.iter_content(8192):
                f.write(chunk)

    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extract(target_file, path=work_dir)

    if not os.path.exists(txt_path):
        raise RuntimeError(f"Failed to extract {target_file}")

    return txt_path

# Usage
glove_path = prepare_glove()
print("GloVe file:", glove_path)

GloVe file: /kaggle/working/glove/glove.6B.100d.txt


In [106]:
def create_embedding_layer(
    tokenizer,
    glove_path: str,
    embedding_dim: int,
    mask_zero: bool = True,
    trainable: bool = False,
    oov_token: str = '[UNK]'
) -> Embedding:
    """
    Build a Keras Embedding layer from a fitted tokenizer and a GloVe file.

    Args:
        tokenizer: a fitted keras.preprocessing.text.Tokenizer
        glove_path: path to a GloVe‑style file (word + embedding_dim floats)
        max_num_words: max vocabulary size (typically tokenizer.num_words)
        embedding_dim: dimensionality of the GloVe vectors
        mask_zero: if True, reserve index 0 for padding (and mask it)
        trainable: if False, freeze the embedding weights
        oov_token: the out‑of‑vocab token (must match tokenizer.oov_token)

    Returns:
        A tf.keras.layers.Embedding instance with pretrained weights.
    """
   
    embeddings_index = {}
    with open(glove_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.rstrip().split(" ")
            word = parts[0]
            coefs = np.asarray(parts[1:], dtype='float32')
            if coefs.shape[0] != embedding_dim:
                continue  # skip any lines that don't match expected dim
            embeddings_index[word] = coefs

    
    vocab_size =  len(tokenizer.word_index) + 1
    embedding_matrix = np.random.normal(
        scale=0.01,
        size=(vocab_size, embedding_dim)
    ).astype('float32')

    
    for word, idx in tokenizer.word_index.items():
        if idx == 0 or idx >= vocab_size:
            continue
        vec = embeddings_index.get(word)
        if vec is not None:
            embedding_matrix[idx] = vec

    # 4) Build the Keras layer
    embedding_layer = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        weights=[embedding_matrix],
        mask_zero=mask_zero,
        trainable=trainable,
        name='pretrained_embedding'
    )
    return embedding_layer


## Phase One

In [64]:
MAX_Q_LEN = max(len(s) for s in q_seqs)
MAX_A_LEN = max(len(s) for s in a_seqs)
MAX_C_LEN   = max(len(s) for s in trunc_c_seqs)
MAX_ENCODER_LEN = max(len(s) for s in q_seqs) + 1 + MAX_C_TRUNC
VOCAB_SIZE  = 20000
EMB_DIM     = 100
UNITS       = 128
BATCH_SIZE  = 64

NameError: name 'trunc_c_seqs' is not defined

In [32]:
q_padded = pad_sequences(q_seqs, maxlen=MAX_Q_LEN, padding='post', truncating='post')
a_padded = pad_sequences(a_seqs, maxlen=MAX_A_LEN, padding='post', truncating='post')

In [33]:
EMB_DIM      = 100
GLOVE_PATH   = prepare_glove()

embedding_layer = create_embedding_layer(
    tokenizer=tokenizer_phase_1,
    glove_path=GLOVE_PATH,
    embedding_dim=EMB_DIM,
    mask_zero=True,      
    trainable=False      
)

I0000 00:00:1745267317.709824      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [34]:
decoder_input  = a_padded[:, :-1]
decoder_target = a_padded[:, 1:]

Xq_tr, Xq_val, Din_tr, Din_val, Dt_tr, Dt_val = train_test_split(
    q_padded, decoder_input, decoder_target,
    test_size=0.1, random_state=42
)

def make_ds(q, d_in, d_tar, batch_size=64):
    mask = tf.cast(tf.not_equal(d_tar, 0), tf.float32)
    ds = tf.data.Dataset.from_tensor_slices(
        ((q, d_in), d_tar, mask)
    )
    return ds.shuffle(2000).batch(batch_size).prefetch(1)

train_ds = make_ds(Xq_tr, Din_tr, Dt_tr, batch_size=BATCH_SIZE)
val_ds   = make_ds(Xq_val, Din_val, Dt_val, batch_size=BATCH_SIZE)

In [35]:
print("Train batches:", tf.data.experimental.cardinality(train_ds).numpy())
print("Val batches:", tf.data.experimental.cardinality(val_ds).numpy())

Train batches: 141
Val batches: 16


**Building the model**

In [36]:
class Seq2SeqLSTM(tf.keras.Model):
    def __init__(self,vocab_size,emb_dim,units,max_q_len,max_a_len,embedding_matrix=None,pad_token_id=0,**kwargs):
        super().__init__(**kwargs)
        self.pad_token_id = pad_token_id


        if embedding_matrix is not None:
            self.embedding = Embedding(vocab_size, emb_dim,weights=[embedding_matrix],trainable=False,mask_zero=True)
        else:
            self.embedding = Embedding(vocab_size, emb_dim,mask_zero=True)

        #units is the vector size of the hidden state
        #return_state if true returns the final h and c
        self.encoder_lstm = LSTM(units, return_state=True, name='encoder_lstm')

        #return sequence returns all the hidden states from h_1 to h_n
        #return sequence is for evaluation
        #return state is for inference because after each token generated we need to feed the model the states again
        self.decoder_lstm = LSTM(units,return_sequences=True,return_state=True, name='decoder_lstm')

        #the layer needed to predict the next word
        self.dense = Dense(vocab_size, activation='softmax', name='decoder_dense')

    def call(self, inputs, training=False):
        encoder_inputs, decoder_inputs = inputs

        x_enc = self.embedding(encoder_inputs)
        _, state_h, state_c = self.encoder_lstm(x_enc, training=training)
        encoder_states = [state_h, state_c]

        x_dec = self.embedding(decoder_inputs)
        dec_outputs, _, _ = self.decoder_lstm(x_dec, initial_state=encoder_states, training=training)
        return self.dense(dec_outputs)


In [37]:
model = Seq2SeqLSTM( vocab_size=vocab_size,
                    emb_dim=100,units=128,
                     max_q_len=MAX_Q_LEN,max_a_len=MAX_A_LEN,embedding_matrix=embedding_matrix,pad_token_id=0)

dummy_q = tf.zeros((1, MAX_Q_LEN), dtype=tf.int32)
dummy_a = tf.zeros((1, MAX_A_LEN-1), dtype=tf.int32)
_ = model((dummy_q, dummy_a))


NameError: name 'embedding_matrix' is not defined

In [ ]:
model.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)
model.summary()

In [ ]:
# !mkdir /content/drive/MyDrive/models

In [ ]:
# MODEL_DIR = get_model_dir()
# CHECKPOINT_PATH = os.path.join(MODEL_DIR, 'seq2seq_lstm_best.keras')

# checkpoint_cb = ModelCheckpoint(
#     filepath=CHECKPOINT_PATH,
#     monitor='val_sparse_categorical_accuracy',
#     save_best_only=True,
#     mode='max',
#     verbose=1
# )
# history = model.fit(
#     train_ds,
#     validation_data=val_ds,
#     epochs=EPOCHS,
#     callbacks=[checkpoint_cb]
# )
# print(f"Best model will be saved to: {CHECKPOINT_PATH}")

## Phase 2 using a transformer

In [107]:
EMB_DIM      = 100
GLOVE_PATH   = prepare_glove()

embedding_layer = create_embedding_layer(
    tokenizer=tokenizer_phase_2,
    glove_path=GLOVE_PATH,
    embedding_dim=EMB_DIM,
    # mask_zero=True,      
    trainable=False      
)

In [108]:
PAD_ID = 0  # your pad token
BATCH_SIZE= 64
def make_ds(enc, decin, dectar, batch_size=BATCH_SIZE):
    
    ds = tf.data.Dataset.from_tensor_slices(((enc, decin), dectar))
    
    
    def add_sample_weight(inputs, target):
        
        
        
        weights = tf.cast(tf.not_equal(target, PAD_ID), tf.float32)
        return inputs, target, weights
    
    return (
        ds
        .shuffle(2000)
        .map(add_sample_weight, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

train_ds = make_ds(enc_tr, decin_tr, dectar_tr)
val_ds   = make_ds(enc_val, decin_val, dectar_val)

print("   • train batches:", tf.data.experimental.cardinality(train_ds).numpy())
print("   • val batches:  ", tf.data.experimental.cardinality(val_ds).numpy())


   • train batches: 141
   • val batches:   16


In [109]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, max_len: int, embed_dim: int, **kwargs):
        super().__init__(**kwargs)
        # self.supports_masking = True
        self.max_len   = max_len
        self.embed_dim = embed_dim

        pos = np.arange(max_len)[:, np.newaxis]                 
        dim = np.arange(embed_dim)[np.newaxis, :]                
        angle_rates = 1.0 / np.power(10000.0, (2 * (dim//2)) / embed_dim)
        angle_rads  = pos * angle_rates                          
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])        
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])        

        
        self.pos_encoding = tf.constant(angle_rads[np.newaxis, ...], dtype=tf.float32)
        

    def call(self, x):
        seq_len = tf.shape(x)[1]
        return x + self.pos_encoding[:, :seq_len, :]

    # def compute_mask(self, inputs, mask=None):
        
    #     return mask

    def get_config(self):
        cfg = super().get_config()
        cfg.update({"max_len": self.max_len, "embed_dim": self.embed_dim})
        return cfg

In [110]:
class Encoder(layers.Layer):
    def __init__(self,embed_dim: int, num_heads: int, ff_dim: int, **kwargs):
        super().__init__(**kwargs)
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embed_dim
        )
        self.layer1 = layers.LayerNormalization()
        self.layer2 = layers.LayerNormalization()
        self.ffn =  tf.keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])

    
    def call(self,pos_matrix, padding_mask, **kwargs):
        att_out= self.attention(query=pos_matrix, value=pos_matrix, key=pos_matrix,attention_mask=padding_mask)
        norm1 = self.layer1(att_out+pos_matrix)
        ff_out = self.ffn(norm1)
        return self.layer2(ff_out)

        

In [111]:
class Decoder(layers.Layer):
    def __init__(self, embed_dim: int, num_heads: int, ff_dim: int, dropout=0.1, **kwargs):
        super().__init__(**kwargs)
        # no supports_masking = True
        self.self_mha  = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.cross_mha = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn       = tf.keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.norm1 = layers.LayerNormalization()
        self.norm2 = layers.LayerNormalization()
        self.norm3 = layers.LayerNormalization()

    def call(self, x, enc_out,
             look_ahead_mask=None,
             padding_mask=None,
             training=False):
        # 1) Decoder self‑attention with your 3D look‑ahead+pad mask
        att1 = self.self_mha(
            query=x, value=x, key=x,
            attention_mask=look_ahead_mask,
            training=training
        )
        out1 = self.norm1(x + att1, training=training)

        
        att2 = self.cross_mha(
            query=out1, value=enc_out, key=enc_out,
            attention_mask=padding_mask,
            training=training
        )
        out2 = self.norm2(out1 + att2, training=training)

        
        ffn_out = self.ffn(out2, training=training)
        return self.norm3(out2 + ffn_out, training=training)


In [112]:
class Seq2SeqTransformer(tf.keras.Model):
    def __init__(self,
                 vocab_size,
                 embed_dim,
                 num_heads,
                 ff_dim,
                 max_enc_in_len,
                 max_a_len,
                 embedding_layer,
                 pad_token_id=0,
                 **kwargs):
        super().__init__(**kwargs)
        if embedding_layer is None:
            raise ValueError("`embedding_layer` must be provided")
        self.token_emb   = embedding_layer
        self.pad_id      = pad_token_id
        self.pos_emb_enc = PositionalEmbedding(max_enc_in_len, embed_dim)
        self.pos_emb_dec = PositionalEmbedding(max_a_len, embed_dim)
        self.encoder     = Encoder(embed_dim, num_heads, ff_dim)
        self.decoder     = Decoder(embed_dim, num_heads, ff_dim)
        self.final_dense = layers.Dense(vocab_size, activation="softmax")

    def create_padding_mask(self, seq):
        mask = tf.equal(seq, self.pad_id)
        return mask[:, None, None, :]

    def create_look_ahead_mask(self, size):
        return 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)

    def call(self, inputs, training=False):
        enc_seq, dec_seq = inputs
        batch_size = tf.shape(enc_seq)[0]
        dec_len    = tf.shape(dec_seq)[1]

        
        enc_padding_mask = self.create_padding_mask(enc_seq)
        

        dec_pad_bool = tf.equal(dec_seq, self.pad_id)

        look2d = self.create_look_ahead_mask(dec_len)
        look2d = tf.cast(look2d, tf.bool)

       
        dec_pad_3d = dec_pad_bool[:, None, :]      
        look3d     = look2d[None, :, :]             
        self_attn_mask = tf.logical_or(dec_pad_3d, look3d)

        enc_x = self.pos_emb_enc(self.token_emb(enc_seq))
        dec_x = self.pos_emb_dec(self.token_emb(dec_seq))

        enc_out = self.encoder(
            enc_x,
            padding_mask=enc_padding_mask,
            training=training
        )

        
        dec_out = self.decoder(
            dec_x,
            enc_out,
            look_ahead_mask=self_attn_mask,
            padding_mask=enc_padding_mask,
            training=training
        )

        return self.final_dense(dec_out)

In [113]:
model = Seq2SeqTransformer(
    vocab_size=tokenizer_phase_2.num_words,
    embed_dim=100,
    num_heads=8,
    ff_dim=512,
    max_enc_in_len=MAX_ENCODER_LEN,
    max_a_len=MAX_A_LEN - 1, 
    embedding_layer=embedding_layer,
    pad_token_id=0,
)

In [114]:
dummy_enc = tf.zeros((1, MAX_ENCODER_LEN), dtype=tf.int32)
dummy_dec = tf.zeros((1, MAX_A_LEN-1), dtype=tf.int32)
_ = model((dummy_enc, dummy_dec), training=False)

# now summary will know every shape and param count
model.summary()
model.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'positional_embedding_2' (of type PositionalEmbedding) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'positional_embedding_3' (of type PositionalEmbedding) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model: "seq2_seq_transformer_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ pretrained_embedding (Embedding)     │ ?                           │       5,732,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ positional_embedding_2               │ ?                           │               0 │
│ (PositionalEmbedding)                │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ positional_embedding_3               │ ?                           │               0 │
│ (PositionalEmbedding)                │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ encoder_1 (Encoder)                  │ ?                           │         425,912 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ decoder_1 (Decoder)                  │ ?                           │         748,612 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (1, 31, 50000)              │       5,050,000 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,956,724 (45.61 MB)

 Trainable params: 6,224,524 (23.74 MB)

 Non-trainable params: 5,732,200 (21.87 MB)

In [122]:
MODEL_DIR = get_model_dir()
CHECKPOINT_PATH = os.path.join(MODEL_DIR, 'seq2seq_transformer_best2.keras')

checkpoint_cb = ModelCheckpoint(
    filepath=CHECKPOINT_PATH,
    monitor='val_sparse_categorical_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=600,
    callbacks=[checkpoint_cb]
)
print(f"Best model will be saved to: {CHECKPOINT_PATH}")

Epoch 1/600
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0099 - sparse_categorical_accuracy: 0.1358
Epoch 1: val_sparse_categorical_accuracy improved from -inf to 0.10463, saving model to /kaggle/working/models/seq2seq_transformer_best2.keras
141/141 ━━━━━━━━━━━━━━━━━━━━ 12s 84ms/step - loss: 0.0099 - sparse_categorical_accuracy: 0.1358 - val_loss: 0.4809 - val_sparse_categorical_accuracy: 0.1046
Epoch 2/600
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0101 - sparse_categorical_accuracy: 0.1347
Epoch 2: val_sparse_categorical_accuracy improved from 0.10463 to 0.10476, saving model to /kaggle/working/models/seq2seq_transformer_best2.keras
141/141 ━━━━━━━━━━━━━━━━━━━━ 12s 84ms/step - loss: 0.0101 - sparse_categorical_accuracy: 0.1347 - val_loss: 0.4701 - val_sparse_categorical_accuracy: 0.1048
Epoch 3/600
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0098 - sparse_categorical_accuracy: 0.1348
Epoch 3: val_sparse_categorical_accuracy did not improve from 0.10476
141/14

In [116]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

def greedy_decode(model,
                  question: str,
                  context: str,
                  tokenizer,
                  max_encoder_len: int,
                  max_answer_len: int,
                  sep_token_id: int,
                  pad_token_id: int,
                  sos_token_id: int,
                  eos_token_id: int):
    q_ids = tokenizer.texts_to_sequences([question])[0]
    c_ids = tokenizer.texts_to_sequences([context])[0]
    enc_seq = q_ids + [sep_token_id] + c_ids
    enc_input = pad_sequences(
        [enc_seq],
        maxlen=max_encoder_len,
        padding='post',
        truncating='post',
        value=pad_token_id
    )

   
    dec_input = [sos_token_id]

    for _ in range(max_answer_len):
        # pad decoder input to max_answer_len
        dec_pad = pad_sequences(
            [dec_input],
            maxlen=max_answer_len,
            padding='post',
            truncating='post',
            value=pad_token_id
        )

        preds = model((enc_input, dec_pad), training=False)
        next_id = int(tf.argmax(preds[0, len(dec_input)-1]).numpy())
        if next_id == eos_token_id:
            break

        dec_input.append(next_id)

    decoded_tokens = [tokenizer.index_word.get(i, "") for i in dec_input[1:]]
    return " ".join(decoded_tokens)


sos_id = tokenizer_phase_2.word_index["[SOS]"]
eos_id = tokenizer_phase_2.word_index["[EOS]"]
sep_id = tokenizer_phase_2.word_index["[SEP]"]
pad_id = 0

question = "What is the capital of France?"
context  = ("France is a country in Western Europe. "
            "Its capital city is Paris, known for the Eiffel Tower.")

answer = greedy_decode(
    model,
    question,
    context,
    tokenizer_phase_2,
    max_encoder_len=MAX_ENCODER_LEN,
    max_answer_len=MAX_A_LEN-1,
    sep_token_id=sep_id,
    pad_token_id=pad_id,
    sos_token_id=sos_id,
    eos_token_id=eos_id
)
print("Answer:", answer)


Answer: charophyta


In [120]:
question = "What is my name?"
context = "My name is ahmed and fuck you. name is ahmed"

answer2 = greedy_decode(
    model,
    question,
    context,
    tokenizer_phase_2,
    max_encoder_len=MAX_ENCODER_LEN,
    max_answer_len=MAX_A_LEN-1,
    sep_token_id=sep_id,
    pad_token_id=pad_id,
    sos_token_id=sos_id,
    eos_token_id=eos_id
)

print("Answer: ", answer2 )

Answer:  charophyta


In [121]:
model.summary()

Model: "seq2_seq_transformer_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ pretrained_embedding (Embedding)     │ ?                           │       5,732,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ positional_embedding_2               │ ?                           │               0 │
│ (PositionalEmbedding)                │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ positional_embedding_3               │ ?                           │               0 │
│ (PositionalEmbedding)                │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ encoder_1 (Encoder)                  │ ?                           │         425,912 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ decoder_1 (Decoder)                  │ ?                           │         748,612 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (1, 31, 50000)              │       5,050,000 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,405,774 (93.10 MB)

 Trainable params: 6,224,524 (23.74 MB)

 Non-trainable params: 5,732,200 (21.87 MB)

 Optimizer params: 12,449,050 (47.49 MB)

## Inference using the best model

In [123]:
from datasets import load_dataset

squad_dev = load_dataset("squad_v2", split="validation")


README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [ ]:
import tensorflow as tf
from datasets import load_dataset
import evaluate
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.preprocessing.sequence import pad_sequences


squad_dev    = load_dataset("squad_v2", split="validation")
squad_metric = evaluate.load("squad_v2")


token_acc = SparseCategoricalAccuracy(name="sparse_token_accuracy")


def decode_and_logits(example):
    q, c = example["question"], example["context"]
    # build encoder input
    q_ids = tokenizer_phase_2.texts_to_sequences([q])[0]
    c_ids = tokenizer_phase_2.texts_to_sequences([c])[0]
    enc = pad_sequences([q_ids + [sep_id] + c_ids],
                        maxlen=MAX_ENCODER_LEN,
                        padding="post",
                        value=pad_id)
    
    dec_input = [sos_id]
    for _ in range(MAX_A_LEN-1):
        dec_pad = pad_sequences([dec_input],
                                maxlen=MAX_A_LEN-1,
                                padding="post",
                                value=pad_id)
        preds = model((enc, dec_pad), training=False)  # (1, tgt_len, vocab)
        next_id = int(tf.argmax(preds[0, len(dec_input)-1]).numpy())
        if next_id == eos_id:
            break
        dec_input.append(next_id)

    pred_text = " ".join(tokenizer_phase_2.index_word.get(i, "")
                         for i in dec_input[1:])

    # --- Sparse accuracy: compare the model’s *prob* distributions
    #     against gold token‑ids if you have them. If you saved gold dev
    #     token‑ids in a_dev_toks:
    #   token_acc.update_state(
    #       y_true=a_dev_toks[1:],       # shape (tgt_len,)
    #       y_pred=preds[0],             # shape (tgt_len, vocab)
    #       sample_weight=(a_dev_toks[1:] != pad_id).astype(float)
    #   )

    return example["id"], pred_text

# 4) Run through the dev set

# 6) If you had gold token sequences for answers, you’d now compute token‐accuracy:
# print(f"Sparse Token Accuracy: {token_acc.result().numpy():.4f}")


In [ ]:
preds   = []
refs    = []
for ex in squad_dev:
    uid, ptxt = decode_and_logits(ex)
    preds.append({"id": uid, "prediction_text": ptxt})
    refs .append({"id": uid, "answers": ex["answers"]})

# 5) Compute EM/F1
results = squad_metric.compute(predictions=preds, references=refs)
print(f"Exact Match: {results['exact_match']:.2f}")
print(f"     F1:      {results['f1']:.2f}")
